In [1]:
import os
import pandas as pd
import tensorflow as tf
import time
from PIL import Image 
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam

In [2]:
def load_images_to_array(folder_path,image_list):
    target_size=(224, 224)
    normalize=True
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if filename.lower().endswith(('.tif', '.tiff')):
            # Load the image using Pillow
            img = Image.open(file_path)

            # Resize the image to the target size
            img = img.resize(target_size)

            # Convert the resized image to a numpy array
            img_array = np.array(img)

            # Normalize the pixel values
            if normalize:
                img_array = img_array / 65535.0

            # Append the array to the list
            image_list.append(img_array)

    # Convert the list of arrays to a numpy array
    image_array = np.stack(image_list)

    return image_array

In [3]:
kidney_scans = []

In [4]:
mask_scans = []

In [5]:
load_images_to_array("C:/Users/youss/OneDrive - aucegypt.edu/Desktop/kidney project/dataset/blood-vessel-segmentation/train/kidney_1_dense/images",kidney_scans)
load_images_to_array("C:/Users/youss/OneDrive - aucegypt.edu/Desktop/kidney project/dataset/blood-vessel-segmentation/train/kidney_1_dense/labels",mask_scans)
#kidney_1_dense

load_images_to_array("C:/Users/youss/OneDrive - aucegypt.edu/Desktop/kidney project/dataset/blood-vessel-segmentation/train/kidney_1_voi/images",kidney_scans)
load_images_to_array("C:/Users/youss/OneDrive - aucegypt.edu/Desktop/kidney project/dataset/blood-vessel-segmentation/train/kidney_1_voi/labels",mask_scans)
#kidney_1_voi


load_images_to_array("C:/Users/youss/OneDrive - aucegypt.edu/Desktop/kidney project/dataset/blood-vessel-segmentation/train/kidney_2/images",kidney_scans)
load_images_to_array("C:/Users/youss/OneDrive - aucegypt.edu/Desktop/kidney project/dataset/blood-vessel-segmentation/train/kidney_2/labels",mask_scans)
#kidney_2

load_images_to_array("C:/Users/youss/OneDrive - aucegypt.edu/Desktop/kidney project/dataset/blood-vessel-segmentation/train/kidney_3_sparse/images",kidney_scans)
load_images_to_array("C:/Users/youss/OneDrive - aucegypt.edu/Desktop/kidney project/dataset/blood-vessel-segmentation/train/kidney_3_sparse/labels",mask_scans)
#kidney_3_sparse

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [6]:
kidneys = np.array(kidney_scans)
masks = np.array(mask_scans)

In [7]:
# Set a random seed for reproducibility
seed = 42
np.random.seed(seed)

# Create an array of indices and shuffle it
indices = np.arange(len(kidneys))
np.random.shuffle(indices)

# Use the shuffled indices to shuffle both arrays
shuffled_kidneys = kidneys[indices]
shuffled_masks = masks[indices]


In [8]:
from sklearn.model_selection import train_test_split

# Split the data into training (90%), dev (5%), and test (5%)
kidneys_train, kidneys_temp, masks_train, masks_temp = train_test_split(
    shuffled_kidneys, shuffled_masks, test_size=0.1, random_state=seed
)

# Further split the temp sets into dev and test
kidneys_dev, kidneys_test, masks_dev, masks_test = train_test_split(
    kidneys_temp, masks_temp, test_size=0.5, random_state=seed
)


In [9]:
kidneys_train.shape

(6235, 224, 224)

In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

# Define the U-Net model
def unet_model(input_shape):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottom
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)

    # Decoder
    up4 = UpSampling2D(size=(2, 2))(conv3)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(up4)
    concat4 = concatenate([conv4, conv2], axis=-1)

    up5 = UpSampling2D(size=(2, 2))(concat4)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up5)
    concat5 = concatenate([conv5, conv1], axis=-1)

    # Output layer
    output = Conv2D(1, (1, 1), activation='sigmoid')(concat5)

    model = Model(inputs=inputs, outputs=output)
    return model

# Create the U-Net model
input_shape = (224,224,1)  # Adjust the dimensions based on your data
unet_binary_segmentation = unet_model(input_shape)

# Compile the model
unet_binary_segmentation.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the summary of the model
unet_binary_segmentation.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 224, 224, 64  640         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 112, 112, 64  0           ['conv2d_6[0][0]']               
                                )                                                           

In [20]:
from tensorflow.keras.callbacks import ModelCheckpoint
# Define ModelCheckpoint to save the best model during training
model_checkpoint = ModelCheckpoint('unet_model_best.h5', save_best_only=True, save_weights_only=True)

# Train the model
batch_size = 16
epochs = 10

history = unet_binary_segmentation.fit(
    kidneys_train, masks_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(kidneys_dev, masks_dev),
    callbacks=[model_checkpoint]
)

Epoch 1/10
390/390 [==============================] - 3874s 10s/step - loss: 0.0052 - accuracy: 0.9833 - val_loss: 3.5964e-04 - val_accuracy: 0.9864
Epoch 2/10
390/390 [==============================] - 3775s 10s/step - loss: 3.7549e-04 - accuracy: 0.9858 - val_loss: 3.5964e-04 - val_accuracy: 0.9864
Epoch 3/10
390/390 [==============================] - 3767s 10s/step - loss: 3.7549e-04 - accuracy: 0.9858 - val_loss: 3.5964e-04 - val_accuracy: 0.9864
Epoch 4/10
390/390 [==============================] - 3790s 10s/step - loss: 3.7549e-04 - accuracy: 0.9858 - val_loss: 3.5964e-04 - val_accuracy: 0.9864
Epoch 5/10
390/390 [==============================] - 3771s 10s/step - loss: 3.7549e-04 - accuracy: 0.9858 - val_loss: 3.5964e-04 - val_accuracy: 0.9864
Epoch 6/10
390/390 [==============================] - 3765s 10s/step - loss: 3.7549e-04 - accuracy: 0.9858 - val_loss: 3.5964e-04 - val_accuracy: 0.9864
Epoch 7/10
390/390 [==============================] - 3761s 10s/step - loss: 3.7549e-0

In [27]:
# Assuming kidneys_test[50] is a single kidney scan
# Add extra dimensions to represent batch size and channels
input_kidney_scan = np.expand_dims(kidneys_test[50], axis=(0, -1))

# Make predictions
predictions = unet_binary_segmentation.predict(input_kidney_scan)

1/1 [==============================] - 0s 445ms/step
